# Rolling back an update to Database

**Using the `Account` class to create bank accounts and store transaction details, you can 'roll back' a transaction in case there is an error.**

**Database table for accounts:**

| <mark>name</mark>  | balance   |
| :----------------: | :-------: |
| John               | 100.00    |
| *String*           | *Integer* |

**Database table for transactions:**

| <mark>time</mark>         | <mark>account</mark> | amount    |
| :-----------------------: | :------------------: |:--------: |
| '2024-03-10 09:03:06.514' | John                 | 10.10     |
| *Datetime*                | *String*             | *Integer* |

**The highlighted column headers are the primary keys. As you can see, the transaction table has two primary keys that work together to become a unique feature, known as a 'composite' primary key. The time of the transaction together with the account holder's name is unique to the transaction event. The account holder's name is also the primary key (unique) in the accounts table.**

**Where the data is saved in the database (see `_save_update()` method), you can add a `rollback()` instruction, that allows for a rollback when there is an SQLite error. The update is protected by `try` and `else` block that rolls back any changes that are pending that raised an error.**

In [1]:
import sqlite3
import datetime
import pytz

In [6]:
class Account(object):
    
    @staticmethod
    def _current_time():
        return pytz.utc.localize(datetime.datetime.utcnow())
    
    def __init__(self, name, opening_balance=0):
        cursor = db.execute("SELECT name, balance FROM accounts WHERE (name=?)", (name, ))
        row = cursor.fetchone()
        
        if row:
            self.name, self._balance = row
            print(f"\tRetrieved account details for {self.name}")
        else:
            self.name = name
            self._balance = opening_balance
            cursor.execute("INSERT INTO accounts VALUES(?, ?)", (name, opening_balance))
            cursor.connection.commit()
            print(f"\tAccount created for {self.name}")
            
        self.show_balance()
    
    def _save_update(self, amount):
        new_balance = self._balance + amount
        deposit_time = Account._current_time()
        
        # Protect update with rollback in case of error
        try:
            db.execute("UPDATE accounts SET balance= ? WHERE (name=?)", (new_balance, self.name))
            db.execute("INSERT INTO transactions VALUES(?, ?, ?)", (deposit_time, self.name, amount))
        except sqlite3.Error:
            db.rollback()
        else:
            db.commit()
            self._balance = new_balance
    
    def deposit(self, amount):
        if amount > 0.0:
            self._save_update(amount)
            print("£{:.2f} deposited".format(amount / 100))
            
        return self._balance / 100
    
    def withdraw(self, amount):
        if 0.0 < amount <= self._balance:
            self._save_update(-amount)
            print("£{:.2f} withdrawn".format(amount / 100))
            return amount / 100
        else:
            print(f"Not enough funds to withdraw £{amount}")
            return 0.0
    
    def show_balance(self):
        print("Balance on account is £{:.2f}".format(self._balance / 100))


In [3]:
db = sqlite3.connect('accounts/accounts.sqlite', detect_types=sqlite3.PARSE_DECLTYPES)

db.execute("CREATE TABLE IF NOT EXISTS accounts (name TEXT PRIMARY KEY NOT NULL, balance INTEGER NOT NULL)")
db.execute("CREATE TABLE IF NOT EXISTS transactions (time TIMESTAMP NOT NULL, account TEXT NOT NULL, amount INTEGER NOT NULL, "
           "PRIMARY KEY (time, account))")

In [7]:
# Insert amounts in pennies

john = Account('John')
john.deposit(2002)
john.withdraw(1010)
john.show_balance()

print('*' * 80)

terry = Account('Terry')
terry.deposit(1)
terry.deposit(1)
terry.deposit(1)
terry.show_balance()

print('*' * 80)

# New account
terryJ = Account('TerryJ')
terryJ.deposit(100)
terryJ.deposit(5)
terryJ.show_balance()

print('*' * 80)

graham = Account('Graham')
graham.withdraw(100)
graham.show_balance()

print('*' * 80)

sally = Account('Sally')
sally.deposit(1000)
sally.show_balance()

print('*' * 80)

# New account
terryG = Account('TerryG')
terryG.deposit(4000)
terryG.withdraw(5000)
terryG.show_balance()

print('*' * 80)

	Retrieved account details for John
Balance on account is £9.92
£20.02 deposited
£10.10 withdrawn
Balance on account is £19.84
********************************************************************************
	Retrieved account details for Terry
Balance on account is £0.03
£0.01 deposited
£0.01 deposited
£0.01 deposited
Balance on account is £0.06
********************************************************************************
	Retrieved account details for TerryJ
Balance on account is £1.05
£1.00 deposited
£0.05 deposited
Balance on account is £2.10
********************************************************************************
	Retrieved account details for Graham
Balance on account is £0.00
Not enough funds to withdraw £100
Balance on account is £0.00
********************************************************************************
	Retrieved account details for Sally
Balance on account is £10.00
£10.00 deposited
Balance on account is £20.00
******************************************

**There are no errors and the accounts are created and the transactions are stored in the database:**

In [8]:
for row in db.execute("SELECT * FROM transactions"):
    print(row)

(datetime.datetime(2024, 3, 15, 8, 59, 17, 52126), 'John', 2002)
(datetime.datetime(2024, 3, 15, 8, 59, 17, 59747), 'John', -1010)
(datetime.datetime(2024, 3, 15, 8, 59, 17, 72719), 'Terry', 1)
(datetime.datetime(2024, 3, 15, 8, 59, 17, 78830), 'Terry', 1)
(datetime.datetime(2024, 3, 15, 8, 59, 17, 85762), 'Terry', 1)
(datetime.datetime(2024, 3, 15, 8, 59, 17, 98602), 'TerryJ', 100)
(datetime.datetime(2024, 3, 15, 8, 59, 17, 104785), 'TerryJ', 5)
(datetime.datetime(2024, 3, 15, 8, 59, 17, 123918), 'Sally', 1000)
(datetime.datetime(2024, 3, 15, 8, 59, 17, 137425), 'TerryG', 4000)
(datetime.datetime(2024, 3, 15, 9, 3, 43, 750467), 'John', 2002)
(datetime.datetime(2024, 3, 15, 9, 3, 43, 757533), 'John', -1010)
(datetime.datetime(2024, 3, 15, 9, 3, 43, 764222), 'Terry', 1)
(datetime.datetime(2024, 3, 15, 9, 3, 43, 769716), 'Terry', 1)
(datetime.datetime(2024, 3, 15, 9, 3, 43, 776355), 'Terry', 1)
(datetime.datetime(2024, 3, 15, 9, 3, 43, 783663), 'TerryJ', 100)
(datetime.datetime(2024, 3, 

In [9]:
db.close()